# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import numpy as np
import glob
import warnings
import scipy as sp
import scipy.io
import scipy.signal
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

warnings.filterwarnings("ignore")

% matplotlib inline
plt.style.use('ggplot')
fs = 125

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(sig, freq_filter=(40/60,240/60), fs=125):
    """Bandpass Filter.
    
    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = scipy.signal.butter(3, freq_filter, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, sig)

def FourierTransform(signal, fs):
    """
    Discrete Fourier Transform on a signal
    Args:
        signal: (np.array) The input signal
        fs: (number) The sampling rate of <signal>
    
    Returns:
        Freq and Magnitude of the signal
    """
    freqs = np.fft.rfftfreq(6 * len(signal), 1 / fs)
    fft = np.abs(np.fft.rfft(signal, 6 * len(signal)))
    return freqs, fft

def CreateFeatures(ppg, accx, accy, accz, fs=125):
    """
    Create features from the data

    Args:
        ppg: (np.array) The input signal
        accx
        accy
        accz
        fs: (number) The sampling rate of <signal>

    Returns:
        PPG and ACC features
    """

    # Filter data
    ppg_freqs, ppg_fft = FourierTransform(ppg, fs)
    ppg_fft[ppg_freqs <= 40 / 60] = 0.0
    ppg_fft[ppg_freqs >= 240 / 60] = 0.0

    acc = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)

    # FFT for acc
    acc_freqs, acc_fft = FourierTransform(acc, fs)
    acc_fft[acc_freqs <= 40 / 60] = 0.0
    acc_fft[acc_freqs >= 240 / 60] = 0.0

    # max freq for ppg
    ppg_feature = ppg_freqs[np.argmax(ppg_fft)]

    # max freq for acc
    acc_feature = acc_freqs[np.argmax(acc_fft)]

    # The mean of each channel
    mn_x = np.mean(accx)
    mn_y = np.mean(accy)
    mn_z = np.mean(accz)

    # The standard deviation of each channel
    std_x = np.std(accx)
    std_y = np.std(accy)
    std_z = np.std(accz)

    # Various percentile values for each channel
    p5_x = np.percentile(accx, 5)
    p5_y = np.percentile(accy, 5)
    p5_z = np.percentile(accz, 5)
    p10_x = np.percentile(accx, 10)
    p10_y = np.percentile(accy, 10)
    p10_z = np.percentile(accz, 10)
    p25_x = np.percentile(accx, 25)
    p25_y = np.percentile(accy, 25)
    p25_z = np.percentile(accz, 25)
    p50_x = np.percentile(accx, 50)
    p50_y = np.percentile(accy, 50)
    p50_z = np.percentile(accz, 50)
    p90_x = np.percentile(accx, 90)
    p90_y = np.percentile(accy, 90)
    p90_z = np.percentile(accz, 90)
    return np.array([ppg_feature, acc_feature, mn_x, mn_y,
                     mn_z, std_x, std_y, std_z, p5_x, p5_y, p5_z, p10_x, p10_y, p10_z
                        , p25_x, p25_y, p25_z, p50_x, p50_y, p50_z, p90_x, p90_y, p90_z])

def RandomForestRegressorModel():
    """
    Train Regressor model

    Returns:
        Trained model
    """
    fs = 125
    win_length = 8
    win_shift = 2

    # Load filenames through LoadTroikaDataset
    data_fls, ref_fls = LoadTroikaDataset()
    features, labels, signals = [], [], []

    for data_fl, ref_fl in zip(data_fls, ref_fls):
        signal = LoadTroikaDataFile(data_fl)
        ref = np.array([_[0] for _ in scipy.io.loadmat(ref_fl)['BPM0']])
        left_s = (np.cumsum(np.ones(min(signal.shape[1],
                                        len(ref))) * fs * win_shift) -
                  fs * win_shift).astype(int)
        right_s = left_s + fs * win_length
        for i in range(len(left_s)):
            left, right = left_s[i], right_s[i]
            ppg = BandpassFilter(signal[0, left:right])
            accx = BandpassFilter(signal[1, left:right])
            accy = BandpassFilter(signal[2, left:right])
            accz = BandpassFilter(signal[3, left:right])

            features.append(CreateFeatures(ppg, accx, accy, accz))
            labels.append(ref[i])
            signals.append([ppg, accx, accy, accz])

    features, labels = np.array(features), np.array(labels)
    model = RandomForestRegressor(n_estimators=50,
                                  max_depth=10, random_state=42,
                                  criterion='mae',
                                  min_samples_split=5)
    for train_idx, test_idx in KFold(n_splits=5).split(features, labels):
        X_train, y_train = features[train_idx], labels[train_idx]
        X_test, y_test = features[test_idx], labels[test_idx]
        model.fit(X_train, y_train)

    return model

def ProcessData(data_fl, ref_fl):
    """
    Process Data
    Args:
        data_fl: data files
        ref_fls: refrence files
    Returns:
        features, labels, signals
    """
    # Load data using LoadTroikaDataFile
    signal = LoadTroikaDataFile(data_fl)
    features, labels, signals = [], [], []
    # Load ground truth heart rate
    ref = np.array([_[0] for _ in scipy.io.loadmat(ref_fl)['BPM0']])
    fs = 125
    win_length = 8
    win_shift = 2
    left_s = (np.cumsum(np.ones(min(signal.shape[1],
                                    len(ref))) * fs * win_shift) -
              fs * win_shift).astype(int)
    right_s = left_s + fs * win_length
    for i in range(len(left_s)):
        left, right = left_s[i], right_s[i]
        ppg = BandpassFilter(signal[0, left:right])
        accx = BandpassFilter(signal[1, left:right])
        accy = BandpassFilter(signal[2, left:right])
        accz = BandpassFilter(signal[3, left:right])
        features.append(CreateFeatures(ppg, accx, accy, accz))
        labels.append(ref[i])
        signals.append([ppg, accx, accy, accz])
    return features, labels, signals

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Process Data
    Args:
        data_fl: data files
        ref_fls: refrence files
    Returns:
        errors, confidence
    """
    features, labels, signals = ProcessData(data_fl, ref_fl)
    features, labels = np.array(features), np.array(labels)
    model = RandomForestRegressorModel()

    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = [], []

    for i in range(len(signals)):
        feature, label = features[i], labels[i]
        ppg, accx, accy, accz = signals[i]
        pred = model.predict(np.reshape(feature, (1, -1)))[0]
        ppg = BandpassFilter(ppg)
        accx = BandpassFilter(accx)
        accy = BandpassFilter(accy)
        accz = BandpassFilter(accz)

        freqs = np.fft.rfftfreq(2 * len(ppg), 1 / fs)
        fft = np.abs(np.fft.rfft(ppg, 2 * len(ppg)))
        fft[freqs <= 40 / 60] = 0.0
        fft[freqs >= 240 / 60] = 0.0

        pred_fs = pred / 55
        pred_fs_win = (freqs >= pred_fs - 0.5) & (freqs <= pred_fs + 0.5)
        confid = np.sum(fft[pred_fs_win]) / np.sum(fft)

        errors.append(np.abs(pred - label))
        confidence.append(confid)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidence
